In [41]:
import re
import pickle
from copy import deepcopy
from pathlib import Path
from typing import Optional, List, Union, Tuple
from pprint import pprint

import pandas as pd

In [42]:
LOG_FILE_PATH = 'access.log.1'
LOG_PICKLE_PATH = 'access.log.1.pckl'

In [56]:
log_0 = '10.11.1.9 2242.lnsigo.mipt.ru - - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd1/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_1 = '10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_2 = '93.158.156.97 2246.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "POST / HTTP/1.1" 502 568 "-" "YaAlice/1.0.0.0 Chrome/19.0.1084.5409 Safari/536.5" "-"'
log_3 = '10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd3/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"'
log_ex = f'{log_0}\n{log_1}\n{log_2}\n{log_3}'
log_ex_1 = f'{log_0}\n{log_1}'
log_ex_2 = f'{log_2}\n{log_3}'

In [65]:
def load_logs(log_file_path: str, row_num: Optional[int] = None, parser: Optional[callable] = None) -> pd.DataFrame:
    with open(log_file_path, 'r') as f:
        log = f.readlines() if not row_num or row_num < 1 else [f.readline() for _ in range(row_num)]
    log_df = pd.DataFrame(data=log, columns=['raw'])
    return(log_df)

In [55]:
#pattern_main = '^|\n(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")\n|$'
#pattern_main = '(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")'
pattern_main = '^(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")$'

In [81]:
def get_item_recursive(search_dict: dict, search_key: 'str') -> Tuple[bool, any]:
    if isinstance(search_dict, dict):
        if search_key in search_dict.keys():
            return True, search_dict[search_key]
        else:
            for nested_object in search_dict.values():
                found, result = get_recursive(nested_object, search_key)
                if found:
                    return found, result
    else:
        return False, None

In [82]:
def get_keys_recursive(search_dict: dict) -> list:
    result = []
    if isinstance(search_dict, dict):
        for key, value in search_dict.items():
            result.append(key)
            result.extend(get_keys_recursive(value))
    return result

In [73]:
class LogDataFrame:
    def __init__(self, config: Optional[dict] = None):
        self._reserved = ['raw', '_regex', '_columns']
        self._config = config or {'raw': {'_regex': None, '_columns': None}}
        self._df = pd.DataFrame(columns=['raw'])  #: Optional[pd.DataFrame] = None
        self._df_unrec = pd.DataFrame(columns=['raw'])  #: Optional[pd.DataFrame] = None

    def __call__(self) -> pd.DataFrame:
        pass

    def _check_in_reserved(self, name: str) -> None:
        if name in self._reserved:
            raise ValueError(f'Name "{name}" is reserved')
            
    def _check_in_columns(self, name: str) -> None:
        if name in self._df.columns:
            raise ValueError(f'Name "{name}" is already used as a column name')
    
    def _apply_config(self, df: pd.DataFrame, raw_column_name: str = 'raw') -> Tuple[pd.DataFrame, pd.DataFrame]:
        df = deepcopy(df)
        df_unrec = pd.DataFrame(columns=['raw'])

        if raw_column_name not in df.columns:
            raise ValueError(f'Name {str(column_name)} was not found in input dataframe indexes')
        else:
            drop_labels = [column for column in df.columns if column != raw_column_name]
            df.drop(labels=drop_labels, axis=1, inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.rename(index=str, columns={raw_column_name: 'raw'})

        return df, df_unrec

    def add(self, add_data: Union[str, list, pd.DataFrame], raw_column_name: str = 'raw') -> None:
        if isinstance(add_data, str):
            add_data = add_data.strip('\n')
            add_data = re.sub('\n+', '\n', add_data)
            add_data = add_data.split('\n')
            add_data = pd.DataFrame(data={'raw': add_data})
        elif isinstance(add_data, list):
            add_data = pd.DataFrame(data={'raw': add_data})
        elif isinstance(add_data, pd.DataFrame):
            pass
        else:
            raise TypeError(f'Wrong type of input data: {type(add_data)}')

        df, df_unrec = self._apply_config(add_data)

        self._df = self._df.append(other=df, ignore_index=True)
        self._df_unrec = self._df_unrec.append(other=df_unrec, ignore_index=True)

    def add_file(self, file_path: str) -> None:
        file_path: Path = Path(file_path).resolve()

        if file_path.is_file():
            with file_path.open('r') as f:
                log_data = f.readlines()
            self.add(log_data)
        else:
            raise FileNotFoundError(f'Not found file: {file_path}')
            
    def _apply(self, column: str, regex: str, columns: Optional[List[str]] = None) -> None:
        
        
            
        

In [89]:
a = {'1', '2', '3'} - {'3', '2', '5'}
print(a)

{'1'}


In [45]:
log_df: LogDataFrame = LogDataFrame()
print(log_df._df)
print(log_df._df_unrec)

Empty DataFrame
Columns: [raw]
Index: []
Empty DataFrame
Columns: [raw]
Index: []


In [46]:
log_df.add(log_ex_1)
print(log_df._df)
print(log_df._df_unrec)

                                                 raw
0  10.11.1.9 2242.lnsigo.mipt.ru - - [17/Dec/2018...
1  10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:0...
Empty DataFrame
Columns: [raw]
Index: []


In [47]:
log_df.add(log_ex_2)
print(log_df._df)
print(log_df._df_unrec)

                                                 raw
0  10.11.1.9 2242.lnsigo.mipt.ru - - [17/Dec/2018...
1  10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:0...
2  93.158.156.97 2246.lnsigo.mipt.ru - [17/Dec/20...
3  10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:0...
Empty DataFrame
Columns: [raw]
Index: []


In [59]:
log_df = LogDataFrame()
log_df.add(['a', 'b', 'c'])

In [69]:
match = re.search(pattern_main, log_ex, flags = re.MULTILINE)
if match:
    print(match.group(0))
    print(match.groups())

10.11.1.9 2242.lnsigo.mipt.ru - [17/Dec/2018:06:17:16 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"
('10.11.1.9', '2242.lnsigo.mipt.ru', '-', '[17/Dec/2018:06:17:16 +0300]', '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1"', '200', '26', '"-"', '"python-requests/2.16.5"', '"-"')


In [57]:
found = re.findall(pattern_main, log_ex, flags = re.MULTILINE)
pprint(found)

[('10.11.1.9',
  '2242.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd2/getUpdates HTTP/1.1"',
  '200',
  '26',
  '"-"',
  '"python-requests/2.16.5"',
  '"-"'),
 ('93.158.156.97',
  '2246.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"POST / HTTP/1.1"',
  '502',
  '568',
  '"-"',
  '"YaAlice/1.0.0.0 Chrome/19.0.1084.5409 Safari/536.5"',
  '"-"'),
 ('10.11.1.9',
  '2242.lnsigo.mipt.ru',
  '-',
  '[17/Dec/2018:06:17:16 +0300]',
  '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd3/getUpdates HTTP/1.1"',
  '200',
  '26',
  '"-"',
  '"python-requests/2.16.5"',
  '"-"')]


In [66]:
log: pd.DataFrame = load_logs(log_file_path=LOG_FILE_PATH, row_num=2)

In [67]:
print(log.head(5))

                                                 raw
0  10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:0...
1  10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:0...


In [68]:
log_str = log.iat[0, 0]
print(log_str)

10.11.1.9 2242.lnsigo.mipt.ru - [16/Dec/2018:06:25:12 +0300] "GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd/getUpdates HTTP/1.1" 200 26 "-" "python-requests/2.16.5" "-"



In [69]:
pattern = r'^(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")$'
match = re.search(pattern, log_str)
if match:
    print(match.groups())

('10.11.1.9', '2242.lnsigo.mipt.ru', '-', '[16/Dec/2018:06:25:12 +0300]', '"GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd/getUpdates HTTP/1.1"', '200', '26', '"-"', '"python-requests/2.16.5"', '"-"')
